# East Africa Time Series 

This notebook converts the zonal stats mean values dataframes from output of land_cover_zonal_stats into time series of the 6 environemntal variables for each admin 1 zone across East Africa for all land covers

In [1]:
import numpy as np
import pandas as pd
import os
import collections

In [2]:
#East Africa Time Series Zonal Stats output datasets
EA_TS_INTERIM = '/home/rgreen/DroughtEDM/Data/EA_TS/Interim/'
EA_TS_FINAL = '/home/rgreen/DroughtEDM/Data/EA_TS/Final/'

In [4]:
#create list of dekads as datetime objects
begin = pd.datetime(2002,7,1)
end = pd.datetime(2016,12,31)
dtrange = pd.date_range(begin, end)
days = list(range(0,5298)) #length of dtrange
daysDF = pd.DataFrame({'Days': days}, index=dtrange)
d = daysDF.index.day - np.clip((daysDF.index.day-1) // 10, 0, 2)*10 - 1 #convert daily SM values to dekadal
dekaddates = daysDF.index.values - np.array(d, dtype="timedelta64[D]")
dekads = daysDF.groupby(dekaddates).mean()
dekads = dekads.index

In [5]:
dekads

DatetimeIndex(['2002-07-01', '2002-07-11', '2002-07-21', '2002-08-01',
               '2002-08-11', '2002-08-21', '2002-09-01', '2002-09-11',
               '2002-09-21', '2002-10-01',
               ...
               '2016-09-21', '2016-10-01', '2016-10-11', '2016-10-21',
               '2016-11-01', '2016-11-11', '2016-11-21', '2016-12-01',
               '2016-12-11', '2016-12-21'],
              dtype='datetime64[ns]', length=522, freq=None)

In [52]:
def transform_ts(in_dir, out_dir):
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains('.csv')
    files = files[tifs]

    


    
    for filename in files: 
        
        orig_ts = pd.read_csv(in_dir+filename)
        data_list = orig_ts.values.tolist()
        ts_data = [item[13:] for item in data_list] 
        admin_zones = [item[3] for item in data_list]
  
        
        for i in orig_ts.index:
            var_chunk = np.array_split(ts_data[i], 6)

            out_df = pd.DataFrame({'SM': var_chunk[0].tolist(),                
                   'TMP': var_chunk[1].tolist(),
                   'P':var_chunk[2].tolist(),
                   'LST':var_chunk[3].tolist(),
                   'NDVI':var_chunk[4].tolist(),
                   'RefET':var_chunk[5].tolist()
                  }, index=pd.to_datetime(dekads))


            out_df.to_csv(out_dir+str(admin_zones[i])+ '_' + os.path.splitext(os.path.basename(filename))[0])
            

In [53]:
transform_ts(EA_TS_INTERIM, EA_TS_FINAL)

In [53]:
ea_grasslands = pd.read_csv(EA_TS_INTERIM + 'ea_grasslands.csv')